In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

C:\Users\tsmar\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [90]:
ComicDataRaw = pd.read_csv("C:/Users/tsmar/dc-wikia-data.csv")

In [91]:
ComicDataRaw.fillna(value=0, axis=0, inplace=True)

In [92]:
ComicData2 = ComicDataRaw.drop(labels=['urlslug', 'page_id', 'GSM', 'FIRST APPEARANCE', 'name'], axis=1) #drop unwanted cols
len(ComicData2)
#ComicData3 = ComicData2.fillna(0, axis=0) #drop rows with NaN values.
ComicAlignment = ComicDataRaw.iloc[:, 4]

In [93]:
len(ComicAlignment)

6896

In [94]:
#we will be prediction the alignment of a superhero based on other data about them. 

In [95]:
ComicData = ComicDataRaw.drop(labels=['ALIGN', 'urlslug', 'page_id', 'GSM', 'FIRST APPEARANCE', 'name'], axis=1)
ComicData = ComicData.fillna(value=0, axis=0)

len(ComicData)

6896

In [96]:
ComicData.head

<bound method NDFrame.head of                    ID         EYE        HAIR                SEX  \
0     Secret Identity   Blue Eyes  Black Hair    Male Characters   
1     Secret Identity   Blue Eyes  Black Hair    Male Characters   
2     Secret Identity  Brown Eyes  Brown Hair    Male Characters   
3     Public Identity  Brown Eyes  White Hair    Male Characters   
4     Secret Identity   Blue Eyes  Black Hair    Male Characters   
5     Public Identity   Blue Eyes  Black Hair  Female Characters   
6     Public Identity   Blue Eyes  Blond Hair    Male Characters   
7     Secret Identity   Blue Eyes  Black Hair    Male Characters   
8     Public Identity   Blue Eyes  Blond Hair  Female Characters   
9     Secret Identity   Blue Eyes  Blond Hair    Male Characters   
10    Secret Identity   Blue Eyes  Blond Hair  Female Characters   
11    Secret Identity   Blue Eyes  Blond Hair    Male Characters   
12    Secret Identity   Blue Eyes    Red Hair  Female Characters   
13    Public Ident

In [97]:
#now, this dataframe needs to be converted into numerical data so that machine learning can be run on it. 
#lets build some dictionaries!!!!
eye_map = {'Blue Eyes' : 1, 'Green Eyes' : 2, 'Brown Eyes' : 3, 'Black Eyes' : 4, 'Red Eyes' : 5, 'White Eyes' : 6,
           'Hazel Eyes' : 7,  'Yellow Eyes' : 8, 'Purple Eyes' : 9, 'Orange Eyes' : 10, 'Pink Eyes' : 11, 'Silver Eyes' : 12,
           'Gold Eyes' : 13, 'Grey Eyes' : 14, 'Photocellular Eyes' : 15, 'Amber Eyes' : 16, 'Violet Eyes':17, 'Auburn Hair':20, 0:0}

hair_map = {'Blue Hair' : 1, 'Green Hair' : 2, 'Brown Hair' : 3, 'Black Hair' : 4, 'Red Hair' : 5, 'White Hair' : 6,
           'Hazel Hair' : 7,  'Yellow Hair' : 8, 'Purple Hair' : 9, 'Orange Hair' : 10, 'Pink Hair' : 11, 'Silver Hair' : 12, 
            'Blond Hair' : 13, 'Strawberry Blond Hair' : 14, 'Gold Hair' : 15, 'Grey Hair' : 16, 'Reddish Brown Hair':17, 'Violet Hair':18,
            'Platinum Blond Hair':19, 'Auburn Hair':20, 0:0}

In [98]:
id_map = {'Secret Identity' : 0, 'Public Identity' : 1, 'Identity Unknown': 2, 0:0}
gender_map = {'Male Characters' : 0, 'Female Characters' : 1, 'Genderless Characters': 2, 'Transgender Characters': 3, 0:0}
alive_map = {'Living Characters' : 0, 'Deceased Characters' : 1, 0:0}
align_map = {'Good Characters' : 0, 'Neutral Characters' : 1, 'Bad Characters' : 2, 'Reformed Criminals' : 3, 0:4}

In [99]:
#I will predict if a character is aligned to Good or Evil or Reformed based on these following data items:
    #1. Gender,
    #2. Eye Color
    #3. is Alive
    #4. Number of Appearences
    #5. Year
    #6. simple hash of their name.
    #7. hair color. 

In [100]:
#first we will turn list of alignments into list of numbers: 
import math
num_align = [align_map[d] for d in ComicAlignment] #these are our labels. 
num_gender = [gender_map[g[3]] for g in ComicData.values]
num_eye_color = [eye_map[c[1]] for c in ComicData.values]
num_isAlive = [alive_map[a[4]] for a in ComicData.values]
num_hair = [hair_map[h[2]] for h in ComicData.values]
num_id = [id_map[p[0]] for p in ComicData.values]

In [101]:
ComicData.values[0]

array(['Secret Identity', 'Blue Eyes', 'Black Hair', 'Male Characters',
       'Living Characters', 3093.0, 1939.0], dtype=object)

In [102]:
ComicData['ID'] = num_id
ComicData['EYE'] = num_eye_color
ComicData['ALIVE'] = num_isAlive
ComicData['HAIR'] = num_hair
ComicData['SEX'] = num_gender

In [115]:
ComicData['APPEARANCES'] = ComicData['APPEARANCES'] / 4
ComicData['YEAR'] = ComicData['YEAR'] / 2
ComicData.head

<bound method NDFrame.head of       ID  EYE  HAIR  SEX  ALIVE  APPEARANCES    YEAR
0      0    1     4    0      0      0.77325  0.9695
1      0    1     4    0      0      0.62400  0.9930
2      0    3     3    0      0      0.39125  0.9795
3      1    3     6    0      0      0.32900  0.9935
4      0    1     4    0      0      0.30925  0.9700
5      1    1     4    1      0      0.30775  0.9705
6      1    1    13    0      0      0.28025  0.9705
7      0    1     4    0      0      0.27375  0.9945
8      1    1    13    1      0      0.26875  0.9845
9      0    1    13    0      0      0.25700  0.9780
10     0    1    13    1      0      0.25700  0.9780
11     0    1    13    0      1      0.24225  0.9700
12     0    1     5    1      0      0.23775  0.9835
13     1    1     3    0      0      0.23775  0.9700
14     1    1     4    1      0      0.23350  0.9690
15     1    1     4    0      0      0.23250  0.9715
16     0    1     3    0      0      0.20075  0.9700
17     0    2   

In [104]:
#so we have our labels: 
len(num_align)

6896

In [234]:
#Now the data is ready! we can go to work!
#lets split up the data into training, validation, testing
TrainSplit = 0.50
ValidSplit = 0.25
TestSplit = 0.25

data_size = 6896
train_len = int(data_size * TrainSplit)
valid_len = int(data_size * ValidSplit)
test_len = int(data_size * TestSplit)

TrainSet = ComicData.values[:train_len]
TrainLabels = num_align[:train_len]

ValidationSet = ComicData.values[train_len+1:train_len+valid_len+1]
ValidationLabels = num_align[train_len+1:train_len+valid_len+1]

TestSet = ComicData.values[train_len + valid_len:]
TestLabels = num_align[train_len + valid_len:]

In [235]:
from tensorflow import keras
import matplotlib.pyplot as plt
from keras.optimizers import Adam
from keras.layers import Input, Dense, Activation, Dropout
from keras.models import Sequential
from keras.models import Model

In [236]:
model = Sequential()
model.add(Dense(32, input_dim=7, activation='relu', init='uniform'))
model.add(Dense(16, activation='relu', init='uniform'))
model.add(Dense(8, activation='relu', init='uniform'))
model.add(Dense(5, activation='softmax', init='uniform'))
model.compile(optimizer=Adam(lr=0.02),loss='categorical_crossentropy',metrics=['accuracy'])

C:\Users\tsmar\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(32, input_dim=7, activation="relu", kernel_initializer="uniform")`
  
C:\Users\tsmar\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(16, activation="relu", kernel_initializer="uniform")`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\tsmar\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(8, activation="relu", kernel_initializer="uniform")`
  after removing the cwd from sys.path.
C:\Users\tsmar\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(5, activation="softmax", kernel_initializer="uniform")`
  """


In [237]:
len(TrainLabels)

3448

In [238]:
TrainSet = np.reshape(TrainSet, (train_len, 7))
TrainLabelSet = []
for t in TrainLabels:
    if t == 0:
        TrainLabelSet.append([1, 0, 0, 0, 0])
    elif t == 1:
        TrainLabelSet.append([0, 1, 0, 0, 0])
    elif t == 2:
        TrainLabelSet.append([0, 0, 1, 0, 0])
    elif t == 3:
        TrainLabelSet.append([0, 0, 0, 1, 0])
    elif t == 4:
        TrainLabelSet.append([0, 0, 0, 0, 1])
ValidationSet = np.reshape(ValidationSet, (valid_len, 7))
ValidLabelSet = []
for t in ValidationLabels:
    if t == 0:
        ValidLabelSet.append([1, 0, 0, 0, 0])
    elif t == 1:
        ValidLabelSet.append([0, 1, 0, 0, 0])
    elif t == 2:
        ValidLabelSet.append([0, 0, 1, 0, 0])
    elif t == 3:
        ValidLabelSet.append([0, 0, 0, 1, 0])
    elif t == 4:
        ValidLabelSet.append([0, 0, 0, 0, 1])
        
TestSet = np.reshape(TestSet, (test_len, 7))
TestLabelSet = []
for t in TestLabels:
    if t == 0:
        TestLabelSet.append([1, 0, 0, 0, 0])
    elif t == 1:
        TestLabelSet.append([0, 1, 0, 0, 0])
    elif t == 2:
        TestLabelSet.append([0, 0, 1, 0, 0])
    elif t == 3:
        TestLabelSet.append([0, 0, 0, 1, 0])
    elif t == 4:
        TestLabelSet.append([0, 0, 0, 0, 1])

In [239]:
model.fit(TrainSet, np.array(TrainLabelSet), epochs=3, verbose=1)

Epoch 1/3
3448/3448 [==============================] - 1s 331us/step - loss: 1.1889 - acc: 0.4927
Epoch 2/3
3448/3448 [==============================] - 0s 45us/step - loss: 1.1523 - acc: 0.4983
Epoch 3/3
3448/3448 [==============================] - 0s 46us/step - loss: 1.1484 - acc: 0.4983


In [240]:
model.fit(ValidationSet, np.array(ValidLabelSet), epochs=3, verbose=1)

Epoch 1/3
1724/1724 [==============================] - 0s 55us/step - loss: 1.1175 - acc: 0.5023
Epoch 2/3
1724/1724 [==============================] - 0s 48us/step - loss: 1.1088 - acc: 0.5151
Epoch 3/3
1724/1724 [==============================] - 0s 47us/step - loss: 1.1155 - acc: 0.5174


In [241]:
scores = model.evaluate(TestSet, np.array(TestLabelSet))
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

1724/1724 [==============================] - 0s 196us/step

acc: 52.09%
